In [2]:
# Installation
# pip install -U sentence-transformers
# pip install feedparser pybtex

import numpy as np
import feedparser
from pybtex.database.input import bibtex
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

class FilterPaper2:
    def __init__(self, bibtex_file='My Library.bib'):
        self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

        # Parse bibtex file once at initialization
        parser = bibtex.Parser()
        bib_data = parser.parse_file(bibtex_file)

        # Extract and clean titles in one pass
        queries = []
        for entry in bib_data.entries.values():
            title = entry.fields['title']
            title = title.replace('{','').replace('}','')

            abstract = entry.fields.get('abstract', '')
            queries.append(title + ' ' + abstract if abstract else title)

        self.query = self.model.encode(queries)
        
    def __call__(self, rss, threshold=0.3):
        # Parse feed and extract titles efficiently
        feed = feedparser.parse(rss)
        self.feed_titles = np.array([entry.title for entry in feed.entries])
        self.dates = np.array([entry.date for entry in feed.entries])
        feed_entries = [entry.title + ' ' + entry.get('summary', '') for entry in feed.entries]

        # Compute scores
        embeddings = self.model.encode(feed_entries)

        # Cosine similarity between query and feed
        self.scores = np.quantile(cosine_similarity(embeddings, self.query), 0.95, axis=1)
        if threshold is not None:
            in_threshold = self.scores >= threshold
            self.feed_titles = self.feed_titles[in_threshold]
            self.scores = self.scores[in_threshold]

        sorted_indices = np.argsort(self.scores)[::-1]

        print(f"Found {len(feed_entries)} papers")
        if threshold is not None:
            print(f"Showing papers with similarity >= {100*threshold:.0f}%\n")

        n_top = 10 if len(self.feed_titles) > 10 else len(self.feed_titles)
        print(f"Top {n_top} papers:")
        for i in range(n_top):
            print(f"\t{i+1}. {self.feed_titles[sorted_indices[i]]}. {self.dates[sorted_indices[i]]} ({100*self.scores[sorted_indices[i]]:.0f}%)")
        
        if len(self.feed_titles) > n_top:
            print(f"\nRest of the papers:")
            for i in range(n_top, len(self.feed_titles)):
                print(f"\t{i+1}. {self.feed_titles[sorted_indices[i]]}. {self.dates[sorted_indices[i]]} ({100*self.scores[sorted_indices[i]]:.0f}%)")

fp = FilterPaper2()

c:\Users\lapis\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# bioRxiv
fp('https://connect.biorxiv.org/biorxiv_xml.php?subject=neuroscience')

Found 30 papers
Showing papers with similarity >= 30%

Top 10 papers:
	1. Striatal Astrocytes Influence Dopamine Dynamics and Behavioral State Transitions. 2024-12-02 (51%)
	2. MEG signals reveal arm posture coding and intrinsic movement plans in parietofrontal cortex. 2024-12-02 (50%)
	3. Decoding task representations that support generalization in hierarchical task. 2024-12-02 (50%)
	4. Synaptic acetylcholine induces sharp wave ripples in the basolateral amygdala through nicotinic receptors. 2024-12-02 (48%)
	5. The VGCC auxiliary subunit α2δ1 is an extracellular GluA1 interactor and regulates LTP, spatial memory, and seizure susceptibility. 2024-12-02 (46%)
	6. Overactivation of prefrontal astrocytes impairs cognition through the metabolic pathway of central kynurenines. 2024-12-02 (45%)
	7. Integrated phenomenology and brain connectivity demonstrate changes in nonlinear processing in jhana advanced meditation. 2024-12-02 (44%)
	8. Connectivity in the human and the monkey brain prob

In [4]:
# Nature
fp('http://www.nature.com/nature/current_issue/rss/')

Found 75 papers
Showing papers with similarity >= 30%

Top 6 papers:
	1. Order matters: neurons in the human brain fire in sequences that encode information. 2024-12-03 (52%)
	2. Individual variability of neural computations underlying flexible decisions. 2024-12-03 (49%)
	3. Design tips for reproducible studies linking the brain to behaviour. 2024-12-03 (38%)
	4. Study design features increase replicability in brain-wide association studies. 2024-12-03 (36%)
	5. In the big data era, prioritize statistical significance in study design. 2024-12-03 (35%)
	6. Author Correction: Gamma frequency entrainment attenuates amyloid load and modifies microglia. 2024-12-03 (30%)


In [5]:
# Science
fp('https://www.science.org/action/showFeed?type=etoc&feed=rss&jc=science')

Found 37 papers
Showing papers with similarity >= 30%

Top 2 papers:
	1. Suppression of neurons in circumventricular organs enables months-long survival without water in thirteen-lined ground squirrels. 2024-11-28T07:00:02Z (35%)
	2. Contemporary hominin locomotor diversity. 2024-11-28T07:00:02Z (32%)


In [6]:
# Cell
fp('https://www.cell.com/cell/current.rss')

Found 16 papers
Showing papers with similarity >= 30%

Top 2 papers:
	1. Mechanisms of memory-supporting neuronal dynamics in hippocampal area CA3. 2024-10-24 (55%)
	2. The multi-stage plasticity in the aggression circuit underlying the winner effect. 2024-10-14 (30%)


In [7]:
# Nature Neuroscience
fp('https://www.nature.com/neuro/current.rss')

Found 8 papers
Showing papers with similarity >= 30%

Top 7 papers:
	1. Simultaneous representations of the past and future. 2024-12-03 (48%)
	2. Self-experience of a negative event alters responses to others in similar states through prefrontal cortex CRF mechanisms. 2024-12-03 (44%)
	3. Focus on single-cell genomics. 2024-12-03 (42%)
	4. Opportunities and challenges of single-cell and spatially resolved genomics methods for neuroscience discovery. 2024-12-03 (39%)
	5. Aversion crosses over. 2024-12-03 (36%)
	6. Applying single-cell and single-nucleus genomics to studies of cellular heterogeneity and cell fate transitions in the nervous system. 2024-12-03 (36%)
	7. Enteric neurons and gut immunity. 2024-12-03 (30%)


In [8]:
# Neuron
fp('https://www.cell.com/neuron/current.rss')

Found 13 papers
Showing papers with similarity >= 30%

Top 10 papers:
	1. Offline hippocampal reactivation during dentate spikes supports flexible memory. 2024-09-19 (55%)
	2. The dorsolateral striatum encodes a temporal basis for the organization of behavior. 2024-11-04 (55%)
	3. Reply Letter to: The dorsolateral striatum encodes a temporal basis for the organization of behavior. 2024-11-20 (55%)
	4. Ventral frontostriatal circuitry mediates the computation of reinforcement from symbolic gains and losses. 2024-09-24 (51%)
	5. Human hippocampus and dorsomedial prefrontal cortex infer and update latent causes during social interaction. 2024-09-24 (48%)
	6. A fast and responsive voltage indicator with enhanced sensitivity for unitary synaptic events. 2024-11-20 (44%)
	7. Ventral hippocampal parvalbumin interneurons gate the acute anxiolytic action of the serotonergic psychedelic DOI. 2024-09-20 (44%)
	8. An amygdalar oscillator coordinates cellular and behavioral rhythms. 2024-09-25 (43%

In [9]:
# Nature Reviews Neuroscience
fp('https://www.nature.com/nrn.rss')

Found 8 papers
Showing papers with similarity >= 30%

Top 7 papers:
	1. Ruminating on replay during the awake state. 2024-11-26 (52%)
	2. Neural circuit basis of pathological anxiety. 2024-11-28 (46%)
	3. Structural neural plasticity evoked by rapid-acting antidepressant interventions. 2024-11-18 (45%)
	4. Structural MRI of brain similarity networks. 2024-11-28 (41%)
	5. An interoceptive circuit regulating energy balance. 2024-11-26 (41%)
	6. Echolocation over long distances. 2024-11-27 (33%)
	7. Innate immune control of synapse development. 2024-11-20 (31%)


In [11]:
# Nature Communications
fp('https://www.nature.com/ncomms.rss')

Found 8 papers
Showing papers with similarity >= 30%

Top 0 papers:


In [12]:
import feedparser

rss = 'https://www.cell.com/cell/current.rss'
# rss = 'https://connect.biorxiv.org/biorxiv_xml.php?subject=neuroscience'
# rss = 'http://www.nature.com/nature/current_issue/rss/'
feed = feedparser.parse(rss)

In [13]:
feed

{'bozo': False,
 'entries': [{'id': 'https://www.cell.com/cell/fulltext/S0092-8674(24)01088-2?rss=yes',
   'title': 'The multi-stage plasticity in the aggression circuit underlying the winner effect',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': 'https://www.cell.com/cell/current.rss',
    'value': 'The multi-stage plasticity in the aggression circuit underlying the winner effect'},
   'links': [{'rel': 'alternate',
     'type': 'text/html',
     'href': 'https://www.cell.com/cell/fulltext/S0092-8674(24)01088-2?rss=yes'}],
   'link': 'https://www.cell.com/cell/fulltext/S0092-8674(24)01088-2?rss=yes',
   'summary': 'Repeated winning increases the readiness to attack and the probability of winning, known as the winner effect. Yan et\xa0al. found that the winner effect is supported by the triple-phase plasticity of VMHvl cells, which are causally linked and triggered by the coactivation of VMHvl and PA cells during winning.',
   'summary_detail': {'type': 't

In [15]:
entry = feed.entries[0]

parts = []

if entry.get('authors'):
    parts.append(', '.join(author.name for author in entry.authors))
    
parts.append(entry.title)

if entry.get('dc_source'):
    parts.append(entry.get('dc_source'))

parts.append(entry.get('updated') or entry.get('date'))
parts.append(entry.link)

msg = '\n'.join(parts) + '\n'
print(msg)

Rongzhen Yan, Dongyu Wei, Avni Varshneya, Lynn Shan, Bing Dai, Hector J. Asencio, Aishwarya Gollamudi, Dayu Lin
The multi-stage plasticity in the aggression circuit underlying the winner effect
Cell 187, 24 (2024)
2024-10-14
https://www.cell.com/cell/fulltext/S0092-8674(24)01088-2?rss=yes

